<a href="https://colab.research.google.com/github/Vinuthna06reddy/VinuthnaReddy_INFO5731_FALL2024/blob/main/INFO5731_Assignment_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
!pip install requests beautifulsoup4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import spacy

def scrape_imdb_reviews(movie_url, num_reviews=1000):
    reviews = []
    headers = {"User-Agent": "Mozilla/5.0"}

    for start in range(0, num_reviews, 25):
        url = f"{movie_url}reviews?ref_=tt_ql_3&start={start}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        review_containers = soup.find_all('div', class_='text show-more__control')
        for review in review_containers:
            reviews.append(review.text)

        if len(reviews) >= num_reviews:
            break

    return reviews[:num_reviews]
movie_url = "https://www.imdb.com/title/tt9603212/reviews/?ref_=tt_ov_ql_2/"
reviews = scrape_imdb_reviews(movie_url)

df = pd.DataFrame(reviews, columns=['review'])
df.to_csv('imdb_reviews.csv', index=False)

print("Reviews collected and saved to 'imdb_reviews.csv'")

def clean_reviews(reviews):
    cleaned_reviews = []
    for review in reviews:
        review = re.sub(r'\s+', ' ', review)
        review = re.sub(r'[^\w\s]', '', review)
        review = review.lower()
        cleaned_reviews.append(review)
    return cleaned_reviews

df = pd.read_csv('imdb_reviews.csv')
df['cleaned_review'] = clean_reviews(df['review'])
df.to_csv('cleaned_imdb_reviews.csv', index=False)
print("Reviews cleaned and saved to 'cleaned_imdb_reviews.csv'")

nlp = spacy.load('en_core_web_sm')

def syntactic_analysis(reviews):
    analyzed_data = []
    for review in reviews:
        doc = nlp(review)
        analyzed_data.append([(token.text, token.pos_, token.dep_) for token in doc])
    return analyzed_data
df['syntactic_analysis'] = syntactic_analysis(df['cleaned_review'])
df.to_csv('analyzed_imdb_reviews.csv', index=False)
print("Syntactic analysis completed and saved to 'analyzed_imdb_reviews.csv'")


Reviews collected and saved to 'imdb_reviews.csv'
Reviews cleaned and saved to 'cleaned_imdb_reviews.csv'
Syntactic analysis completed and saved to 'analyzed_imdb_reviews.csv'


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv('analyzed_imdb_reviews.csv')

def remove_noise(text):
    return re.sub(r'[^\w\s]', '', text)
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])
def lowercase(text):
    return text.lower()
stemmer = PorterStemmer()
def stemming(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])
lemmatizer = WordNetLemmatizer()
def lemmatization(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df['cleaned_text'] = df['cleaned_review'].apply(remove_noise)
df['cleaned_text'] = df['cleaned_text'].apply(remove_numbers)
df['cleaned_text'] = df['cleaned_text'].apply(remove_stopwords)
df['cleaned_text'] = df['cleaned_text'].apply(lowercase)
df['cleaned_text'] = df['cleaned_text'].apply(stemming)
df['cleaned_text'] = df['cleaned_text'].apply(lemmatization)

df.to_csv('final_cleaned_imdb_reviews.csv', index=False)

print(df[['cleaned_review', 'cleaned_text']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                      cleaned_review  \
0  man i wish i loved this movie more than i did ...   
1  just got out of seeing this movie i am really ...   
2  just saw this one in a theatre and boy what an...   
3  to me a new mi film is an event from part 4 on...   
4  good god i feel ashamed for approaching this f...   

                                        cleaned_text  
0  man wish love movi dont get wrong solid action...  
1  got see movi realli disappoint total kill pote...  
2  saw one theatr boy adrenalin rush latest missi...  
3  new mi film event part onward theyv spectacula...  
4  good god feel asham approach film modicum doub...  


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
!pip install spacy textblob nltk
!python -m spacy download en_core_web_sm
import spacy
from textblob import TextBlob
from collections import Counter
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")

cleaned_texts = [
    "man wish love movi dont get wrong solid action"
    "got see movi realli disappoint total kill potential"
    "saw one theatr boy adrenalin rush latest missiion impossible"
    "new mi film event part onward theyv spectacular"
    "good god feel asham approach film modicum doubt"
]
pos_counts = Counter({'Nouns': 0, 'Verbs': 0, 'Adjectives': 0, 'Adverbs': 0})

def get_pos_category(tag):
    if tag.startswith('N'):
        return 'Nouns'
    elif tag.startswith('V'):
        return 'Verbs'
    elif tag.startswith('J'):
        return 'Adjectives'
    elif tag.startswith('R'):
        return 'Adverbs'
    return None
for text in cleaned_texts:
    blob = TextBlob(text)
    tags = blob.tags
    for word, pos in tags:
        category = get_pos_category(pos)
        if category:
            pos_counts[category] += 1

print("POS Tagging Counts:", pos_counts)

def analyze_parsing(text):
    doc = nlp(text)
    print(f"\nAnalyzing the text: '{text}'")
    print("Constituency Parsing (approximated as dependency parsing):", doc)
    print("\nDependency Parsing:")
    for token in doc:
        print(f"{token.text} --> {token.dep_} ({token.head.text})")
    print("\nDependency Tree:")
    for token in doc:
        print(f"{token.text} ({token.dep_}) <-- {token.head.text} ({token.head.dep_})")
example_text = cleaned_texts[0]
analyze_parsing(example_text)
ner_counts = Counter()

print("\nNamed Entity Recognition Results:")
for text in cleaned_texts:
    doc = nlp(text)
    for ent in doc.ents:
        ner_counts[ent.label_] += 1
        print(f"Entity: {ent.text}, Label: {ent.label_}")
print("\nNER Entity Counts:", ner_counts)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


POS Tagging Counts: Counter({'Nouns': 24, 'Adjectives': 6, 'Verbs': 5, 'Adverbs': 0})

Analyzing the text: 'man wish love movi dont get wrong solid actiongot see movi realli disappoint total kill potentialsaw one theatr boy adrenalin rush latest missiion impossiblenew mi film event part onward theyv spectaculargood god feel asham approach film modicum doubt'
Constituency Parsing (approximated as dependency parsing): man wish love movi dont get wrong solid actiongot see movi realli disappoint total kill potentialsaw one theatr boy adrenalin rush latest missiion impossiblenew mi film event part onward theyv spectaculargood god feel asham approach film modicum doubt

Dependency Parsing:
man --> nsubj (wish)
wish --> ROOT (wish)
love --> compound (movi)
movi --> nsubj (get)
do --> aux (get)
nt --> neg (get)
get --> ccomp (wish)
wrong --> amod (actiongot)
solid --> amod (actiongot)
actiongot --> intj (see)
see --> conj (wish)
movi --> compound (realli)
realli --> compound (disappoint)
disap

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

https://myunt-my.sharepoint.com/personal/vinuthnavintha_my_unt_edu/Documents/final_cleaned_imdb_reviews.csv?web=1

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
This assignment was a well-rounded introduction to web scraping, text data cleaning, and syntactic analysis, which are important skills in natural language processing (NLP).
Though, this assignment took more time as it required learning to scrape websites, handle dynamic content, clean unstructured text, and perform NLP tasks using libraries like spacy.